#DH

In [8]:
import time
import random
import hashlib
import os
import binascii
import string

from cryptography.hazmat.primitives import asymmetric, hashes, kdf, ciphers, padding
from cryptography.hazmat.backends import default_backend

print("Librerías necesarias importadas.")

def generate_random_text_file(num_words, filename):
    def _generate_word():
        length = random.randint(3, 10)
        return ''.join(random.choice(string.ascii_lowercase) for _ in range(length))

    words = [_generate_word() for _ in range(num_words)]
    with open(filename, 'w') as f:
        f.write(' '.join(words))
    print(f"Se generaron {num_words} palabras aleatorias y se guardaron en '{filename}'.")

print("Función auxiliar 'generate_random_text_file' definida.")

Librerías necesarias importadas.
Función auxiliar 'generate_random_text_file' definida.


In [9]:
word_counts = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
file_paths = {}

print("Verificando y generando archivos de texto para todos los tamaños...")
for num_words in word_counts:
    filename = f"input_shared_{num_words}.txt"
    file_paths[num_words] = filename

    if not os.path.exists(filename):
        generate_random_text_file(num_words, filename)
    else:
        print(f"El archivo '{filename}' ya existe. Se utilizará el archivo existente.")

print("Archivos de texto listos para usar.")

Verificando y generando archivos de texto para todos los tamaños...
El archivo 'input_shared_10.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_100.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_1000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_10000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_100000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_1000000.txt' ya existe. Se utilizará el archivo existente.
El archivo 'input_shared_10000000.txt' ya existe. Se utilizará el archivo existente.
Archivos de texto listos para usar.


In [11]:
import pandas as pd
import time
import os

from cryptography.hazmat.primitives import ciphers, hashes, padding
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend

dh_aes_results = []

print("Iniciando la medición de rendimiento de Diffie-Hellman con AES...")

for num_words in word_counts:
    filename = file_paths[num_words]

    start_time = time.time()
    with open(filename, 'r') as f:
        plaintext_str = f.read()
    plaintext = plaintext_str.encode('utf-8')
    T_E1 = time.time() - start_time

    start_time = time.time()

    parameters = dh.generate_parameters(generator=2, key_size=1024, backend=default_backend())

    alice_private_key = parameters.generate_private_key()
    alice_public_key = alice_private_key.public_key()

    bob_private_key = parameters.generate_private_key()
    bob_public_key = bob_private_key.public_key()

    print(f"  Clave pública de Alice (DH): {hex(alice_public_key.public_numbers().y)}")
    print(f"  Clave pública de Bob (DH): {hex(bob_public_key.public_numbers().y)}")

    alice_shared_key = alice_private_key.exchange(bob_public_key)

    salt = os.urandom(16)
    info = b"handshake data"

    kdf_obj = HKDF(algorithm=hashes.SHA256(), length=32, salt=salt, info=info, backend=default_backend())
    aes_key = kdf_obj.derive(alice_shared_key)

    iv = os.urandom(16)

    print(f"  Clave AES derivada (DH): {aes_key.hex()}")

    T_E2 = time.time() - start_time

    start_time = time.time()
    cipher = ciphers.Cipher(ciphers.algorithms.AES(aes_key), ciphers.modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(ciphers.algorithms.AES.block_size).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    T_E3 = time.time() - start_time

    start_time = time.time()
    decryptor = cipher.decryptor()
    unpadder = padding.PKCS7(ciphers.algorithms.AES.block_size).unpadder()
    decrypted_padded_text = decryptor.update(ciphertext) + decryptor.finalize()
    decrypted_text_bytes = unpadder.update(decrypted_padded_text) + unpadder.finalize()

    T_E4 = time.time() - start_time

    T_Total = T_E1 + T_E2 + T_E3 + T_E4

    input_chars = len(plaintext_str)
    output_chars = len(ciphertext)

    dh_aes_results.append({
        "algoritmo": "DH_AES",
        "#palabras": num_words,
        "#caracteres_entrada": input_chars,
        "#caracteres_salida": output_chars,
        "T-E1": T_E1,
        "T-E2": T_E2,
        "T-E3": T_E3,
        "T-E4": T_E4,
        "T_Total": T_Total
    })

    print(f"Procesamiento completado para {num_words} palabras.")

print("Resultados de rendimiento de Diffie-Hellman con AES:")
df_dh_results = pd.DataFrame(dh_aes_results)
display(df_dh_results)
print("Medición de rendimiento de Diffie-Hellman con AES completada.")

Iniciando la medición de rendimiento de Diffie-Hellman con AES...
  Clave pública de Alice (DH): 0x14c917ef8113367007d5fa61de4bf22db6364beb8dd0ff7328cb0700cde4d57ff3f09be9021d157575faa7ec2dbaf02088b809580254f49a822c0af3209ff439ff759a5a67a358411dbe16348660b385ab62b6f61f824f2836ccd8e26103b3c7cebf704ce4e95a647f2e6e2d0f657ff44fce94c6c1f443a701208f2e7b48cf72
  Clave pública de Bob (DH): 0x2816df06c5733f539d46bdd57d438016d158b5ead7941960ebc3b50b1b2f9b716eea6baf2c223b488e20515a1ac3aa85287efcba42ff9da41e6fe55bf69a35e2a456411cf4674defcdd4cf2ea959ce6dd938eaa73057f674f2306fd18939b52d68c16093f0b6cd7b0cdfac5ea5bb86cf7b0ba21c176e58690dfdc22ef3568f2d
  Clave AES derivada (DH): dab4c3ceb785855eebbc519de890c39d04fb765dbecb75968261d1f3b0f97a98
Procesamiento completado para 10 palabras.
  Clave pública de Alice (DH): 0x64acf78180afb5db7b7574d592c2b42af559191fc996b92097698fb4fe757096b0fbe67da6951149391edb6944218946f6da932df6dd2d17f450c6533233d9c4cb9fd17e5f4424576939a4609ee5d339aadcdbf2b085eb03a04576d82a35

,algoritmo,#palabras,#caracteres_entrada,#caracteres_salida,T-E1,T-E2,T-E3,T-E4,T_Total
0,DH_AES,10,70,80,0.007377,4.524332,0.004758,0.004303,4.540770
1,DH_AES,100,754,768,0.000120,0.467426,0.000134,0.000050,0.467730
2,DH_AES,1000,7512,7520,0.000131,0.373821,0.000101,0.000035,0.374088
3,DH_AES,10000,74568,74576,0.000144,2.280244,0.000269,0.000178,2.280835
4,DH_AES,100000,748950,748960,0.001315,0.294458,0.002602,0.001422,0.299797
5,DH_AES,1000000,7499596,7499600,0.013893,5.232103,0.035128,0.023347,5.304471
6,DH_AES,10000000,74997531,74997536,0.263056,3.480326,0.347390,0.377871,4.468643


Medición de rendimiento de Diffie-Hellman con AES completada.
